In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [4]:
# Hyper Parameters
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [5]:
# MNIST Dataset 
train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),  
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

In [6]:
# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)


In [7]:
# Neural Network Model(1 Hidden Layer)
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [8]:
net = Net(input_size, hidden_size, num_classes)

In [9]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [10]:
# Train the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

Epoch [1/5], Step [100/600], Loss: 0.3338
Epoch [1/5], Step [200/600], Loss: 0.1499
Epoch [1/5], Step [300/600], Loss: 0.1201
Epoch [1/5], Step [400/600], Loss: 0.1773
Epoch [1/5], Step [500/600], Loss: 0.1584
Epoch [1/5], Step [600/600], Loss: 0.1682
Epoch [2/5], Step [100/600], Loss: 0.0994
Epoch [2/5], Step [200/600], Loss: 0.1209
Epoch [2/5], Step [300/600], Loss: 0.3091
Epoch [2/5], Step [400/600], Loss: 0.1529
Epoch [2/5], Step [500/600], Loss: 0.1691
Epoch [2/5], Step [600/600], Loss: 0.0536
Epoch [3/5], Step [100/600], Loss: 0.0440
Epoch [3/5], Step [200/600], Loss: 0.0451
Epoch [3/5], Step [300/600], Loss: 0.0515
Epoch [3/5], Step [400/600], Loss: 0.0538
Epoch [3/5], Step [500/600], Loss: 0.0731
Epoch [3/5], Step [600/600], Loss: 0.0953
Epoch [4/5], Step [100/600], Loss: 0.1276
Epoch [4/5], Step [200/600], Loss: 0.0412
Epoch [4/5], Step [300/600], Loss: 0.0862
Epoch [4/5], Step [400/600], Loss: 0.0698
Epoch [4/5], Step [500/600], Loss: 0.0849
Epoch [4/5], Step [600/600], Loss:

In [13]:
# Test the Model
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28))
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 98 %


In [14]:
# Save the Model
torch.save(net.state_dict(), 'model.pkl')